<a href="https://colab.research.google.com/github/zeqkik/LLM-Fine-Tuning/blob/main/fine_tuning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
import time
import google.generativeai as genai
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.mount('/content/drive')

## Avaliação do Modelo Base

In [ ]:
hf_token = userdata.get('HUGGINGFACE_TOKEN')
login(hf_token)

In [ ]:
model_id = "google/gemma-7b-it"

tokenizer_gemma = AutoTokenizer.from_pretrained(model_id)

model_gemma = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
)


In [ ]:
def generate_response(prompt_text):
  input_ids = tokenizer_gemma(prompt_text, return_tensors="pt").to(model_gemma.device)

  outputs = model_gemma.generate(
        **input_ids,
        max_new_tokens=50,
    )

  response = tokenizer_gemma.decode(outputs[0], skip_special_tokens=True)
  return response


In [ ]:
prompt_1 = """
Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?
"""

In [ ]:
print(generate_response(prompt_1))


Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?
A. Azul 3
B. Verde 5
C. Vermelho 8
D. +2 Amarelo

A resposta para a questão é a carta Vermelho 8.

A carta no descarte é Vermelho 


In [ ]:
prompt_2 = """
Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
"""

In [ ]:
print(generate_response(prompt_2))


Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
**Resposta:** A carta que você podem jogar é a carta Verde 2, pois a regra do Veretchup + 3 exige que o jogador jogue uma carta de mesmo nome que foi lançada no início do jogo.


In [ ]:
prompt_3 = """
Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
"""

In [ ]:
print(generate_response(prompt_3))


Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
**Resposta:** O Verde 2.

**Explicação:**

No momento em que a carta Azul foi descartada, a regra do "Uno" exige que o próximo carta seja de mesmo número ou de mesma cor. Na mão do jogador,


## Geração do Conjunto de Dados

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
api_key = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=api_key)

In [ ]:
uno_rules = """
**Regras Oficiais do Jogo UNO:**
1.  **Objetivo:** Livrar-se de todas as cartas.
2.  **Jogada Válida:** Uma carta só pode ser jogada se combinar com a cor, o número ou o símbolo da carta que está no topo do monte de descarte.
3.  **Carta de Número (0-9):** Combina cor ou número.
4.  **Carta '+2':** Combina cor. O próximo jogador deve comprar duas cartas e perde a vez.
5.  **Carta 'Bloquear' (Skip):** Combina cor. O próximo jogador perde a vez.
6.  **Carta 'Inverter' (Reverse):** Combina cor. Inverte o sentido do jogo.
7.  **Carta 'Trocar Cor' (Wild Card):** Pode ser jogada a qualquer momento, independentemente da carta no descarte. O jogador que a joga escolhe a próxima cor (vermelho, azul, verde ou amarelo).
8.  **Carta '+4' (Wild Draw Four):** Pode ser jogada a qualquer momento, *mas apenas se o jogador não tiver nenhuma carta da cor atual no descarte* (exceto Coringas normais). O jogador que a joga escolhe a próxima cor. O próximo jogador deve comprar quatro cartas e perde a vez.
9.  **Comprar Cartas:** Se um jogador não tiver uma carta válida para jogar, ele deve comprar uma carta do baralho de compra. Se a carta comprada for jogável, ele pode jogá-la imediatamente. Caso contrário, a vez passa para o próximo jogador.
10. **Aviso 'UNO!':** Quando um jogador tem apenas uma carta na mão, ele deve dizer "UNO!". Se ele não disser e outro jogador o pegar antes da vez do próximo jogador, ele deve comprar duas cartas.
"""

In [ ]:
gemini_table_generation_prompt = f"""
Você é um especialista em jogos de cartas, especificamente em UNO. Sua tarefa é criar uma tabela contendo situações de jogo de UNO (entradas) e as jogadas/resultados corretos para cada situação (saídas), baseando-se estritamente nas regras oficiais do jogo que serão fornecidas.

**Objetivo:** Gerar um conjunto de dados para treinar uma IA a jogar UNO, cobrindo a maior variedade possível de situações e regras.

---

{uno_rules}

---

**Instruções Detalhadas para a Geração da Tabela:**

1.  **Formato da Tabela:** Gere uma tabela com **10 exemplos**. Cada exemplo deve ter 2 colunas:
    * **Cenário (prompt):** Uma descrição concisa da situação do jogo. Siga este formato *exato*: `Carta no descarte: [COR/TIPO], Cartas na mão: [[COR/TIPO], [COR/TIPO], ...]. Qual(is) carta(s) da mão você pode jogar ou o que deve fazer?`
    * **Resposta (completion):** A jogada ou o resultado correto para o cenário, **extremamente conciso**. Se houver uma breve justificativa de uma frase (ex: "Combina cor/número", "Causa compra de cartas"), inclua-a após a jogada/resultado. **Não inclua qualquer frase introdutória.**

2.  **Variedade de Cenários:** Garanta que os exemplos abordem uma AMPLA VARIEDADE de situações e testem **diferentes regras** listadas acima.

3.  **Formato da Tabela de Saída (Exemplo):**
    ```
    Cenário (prompt) | Resposta (completion)
    --- | ---
    Carta no descarte: Vermelho 7, Cartas na mão: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo]. Qual(is) carta(s) da mão você pode jogar ou o que deve fazer? | **Vermelho 8**. Combina com a cor Vermelha.
    Carta no descarte: Azul +2, Cartas na mão: [Verde 0, Azul 1, Bloquear Vermelho, Coringa]. O que acontece com você nesta rodada? | Você deve **comprar duas cartas e perder a sua vez**.
    ```
    Gere a tabela completa agora com 250 exemplos, seguindo todas as instruções.
"""

In [ ]:
model_gemini_pro = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
colors = ['Vermelho', 'Azul', 'Verde', 'Amarelo']
numbers = [str(i) for i in range(10)]
action_cards = ['+2', 'Bloquear', 'Inverter']
wild_cards = ['Coringa', '+4']



In [ ]:
full_uno_deck = []
for color in colors:
    full_uno_deck.append(f"{color} 0") # Um "0" de cada cor
    for num in numbers[1:]: # Dois de cada número de 1 a 9
        full_uno_deck.append(f"{color} {num}")
        full_uno_deck.append(f"{color} {num}")

# Cartas de Ação (Duas de cada cor para +2, Bloquear, Inverter)
for color in colors:
    for action in action_cards:
        full_uno_deck.append(f"{color} {action}")
        full_uno_deck.append(f"{color} {action}")

# Cartas Coringa (Quatro de cada)
for _ in range(4):
  for wild_card in wild_cards:
    full_uno_deck.append(f"{wild_card}")


In [ ]:
def generate_random_hand_and_discard(deck):
  remaining_deck = list(deck)
  num_cards = random.randint(3,7)

  hand = random.sample(deck, num_cards)

  for card in hand:
    remaining_deck.remove(card)

  discard_card = random.choice(remaining_deck)
  return hand, discard_card

In [ ]:
def get_gemini_completion_for_prompt(model, prompt_input_text):
  full_gemini_prompt =f"""
    Você é um mestre de Uno e deve analisar um cenário de jogo e determinar a jogada correta ou o resultado, estritamente de acordo com as regras padrão do jogo Uno fornecidas abaixo. Mantenha a resposta com um MÁXIMO de 20-30 palavras, sendo o mais direto possível.

    {uno_rules}

    **Cenário de Jogo:**
    {prompt_input_text}

    **Responda de forma extremamente concisa, indicando apenas a jogada ou o resultado correto. Se houver uma breve justificativa de uma frase, inclua-a após a jogada/resultado. Não inclua qualquer frase introdutória como 'Jogada Correta:' ou 'Resultado:'**
    """
  response_gen = model.generate_content(full_gemini_prompt)

  return response_gen.text.strip()

In [ ]:
num_samples = 450
data_hands = []
data_discards = []
for _ in range(num_samples):
  hand, discard = generate_random_hand_and_discard(full_uno_deck)
  data_hands.append(hand)
  data_discards.append(discard)

In [ ]:
synthetic_dataset = []


In [ ]:
for i in range(num_samples):
  current_hand = data_hands[i]
  current_discard = data_discards[i]

  currente_hand_str = "[" + ",".join(current_hand)+"]"

  question = random.choice(["Qual carta você pode jogar?", "O que você pode fazer?", "O que acontece com você nesta rodada?",])

  prompt_input = f"Carta no descarte: {current_discard}, Cartas na mão: {currente_hand_str}. {question}"

  completion_from_gemini = get_gemini_completion_for_prompt(model_gemini_pro, prompt_input)
  synthetic_dataset.append({
        "prompt": prompt_input, # Este é o prompt limpo que o modelo fine-tuned verá
        "completion": completion_from_gemini # Este é o output gerado pelo Gemini
    })
  time.sleep(2)

In [ ]:
len(synthetic_dataset)

507

In [ ]:
synthetic_dataset

[{'prompt': 'Carta no descarte: Amarelo 7, Cartas na mão: [Coringa,Verde 7,Vermelho Bloquear,Vermelho 3,Amarelo 7,Amarelo 4]. O que você pode fazer?',
  'completion': 'Jogue o Amarelo 7.'},
 {'prompt': 'Carta no descarte: Amarelo 7, Cartas na mão: [Coringa,Verde 7,Vermelho Bloquear,Vermelho 3,Amarelo 7,Amarelo 4]. O que você pode fazer?',
  'completion': 'Amarelo 7.  Combina com a carta descartada.'},
 {'prompt': 'Carta no descarte: Amarelo 7, Cartas na mão: [Coringa,Verde 7,Vermelho Bloquear,Vermelho 3,Amarelo 7,Amarelo 4]. Qual carta você pode jogar?',
  'completion': 'Amarelo 7.  Combina com a cor e o número da carta descartada.'},
 {'prompt': 'Carta no descarte: Azul 8, Cartas na mão: [Coringa,Vermelho 8,Amarelo +2,Verde 7,Vermelho 3,Vermelho 9,Amarelo 4]. Qual carta você pode jogar?',
  'completion': 'Vermelho 8.  Combina com o número 8 no descarte.'},
 {'prompt': 'Carta no descarte: Verde +2, Cartas na mão: [Amarelo 1,Verde 1,Azul 0]. O que você pode fazer?',
  'completion': 'Jog

In [ ]:
df = pd.DataFrame(synthetic_dataset)

In [ ]:
df

,prompt,completion
0,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Jogue o Amarelo 7.
1,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Amarelo 7. Combina com a carta descartada.
2,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Amarelo 7. Combina com a cor e o número da ca...
3,"Carta no descarte: Azul 8, Cartas na mão: [Cor...",Vermelho 8. Combina com o número 8 no descarte.
4,"Carta no descarte: Verde +2, Cartas na mão: [A...",Jogue o Verde 1.
...,...,...
502,"Carta no descarte: Verde 2, Cartas na mão: [Ve...",Jogar o Vermelho 2. Combina com o número da c...
503,"Carta no descarte: Amarelo Inverter, Cartas na...",Comprar carta. Não há cartas na mão que combi...
504,"Carta no descarte: Verde Bloquear, Cartas na m...",Jogue o Amarelo +2.
505,"Carta no descarte: Vermelho 2, Cartas na mão: ...",Vermelho 4. Combina com o número 2 no descarte.


In [ ]:
#df.to_csv('/content/drive/MyDrive/Estudos/LLM_Fine-tuning/training_data.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Estudos/LLM_Fine-tuning/training_data.csv', index_col=0)

# Fine-Tuning

In [ ]:
# Dataset Treatment

raw_dataset = Dataset.from_pandas(df)
train_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)['train']
eval_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)['test']

# 5. FORMATANDO DADOS PARA SFTTrainer
def formatting_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['prompt'])):
        # Formato que o modelo vai aprender a gerar
        text = f"{examples['prompt'][i]}\n{examples['completion'][i]}{tokenizer.eos_token}"
        output_texts.append(text)
    return {"text": output_texts}

# Aplicar formatação
train_dataset_formatted = train_dataset.map(formatting_prompts_func, batched=True)
eval_dataset_formatted = eval_dataset.map(formatting_prompts_func, batched=True)


In [ ]:
# LORA Configuration

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)
model_gemma = get_peft_model(model_gemma, lora_config)




In [ ]:
# Training configuration

training_arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/Estudos/LLM_Fine-tuning/models/gemma_uno_finetuned",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    save_strategy="epoch",
    report_to="none",
    eval_strategy="epoch",
    fp16=True,
)

trainer = SFTTrainer(
    model=model_gemma,
    train_dataset=train_dataset_formatted,
    eval_dataset=eval_dataset_formatted,
    args=training_arguments,
)

trainer.train()